# Double Pendulum Analysis
Is it possible to use a sequence of measurements with known error to reduce your error on the initial measurement? This should be more effective with a chaotic system.

First we will explore this with a simple linear system, then try it with a double pendulum.

In [1]:
# authored by Bryce Burgess
# 27/9/2018
# simulate a double pendulum
# copy that pendulum with error
# see how the initial error can be reduced with subsequent measurements
# can use error threshold on the pendulum (easier), or periodic(more useful)

import matplotlib.animation as ani
import matplotlib.pyplot as plt
import numpy as np
from math import pi, sin, cos

In [25]:
measure_error = 50
slope_error = 0.1
position_error = 0.3

dt = 0.1

line_true_slope = 1
line_guess1_slope = line_true_slope + slope_error
line_guess2_slope = line_true_slope - slope_error

line_true_pos = 0
line_guess1_pos = line_true_pos
line_guess2_pos = line_true_pos

for t in range(100):
    line_true_pos = line_true_slope * t**2
    line_guess1_pos = line_guess1_slope * t**2
    line_guess2_pos = line_guess2_slope * t**2

    #measure
    if t%10 ==0:
        if abs(line_true_pos - line_guess1_pos) > measure_error:
            print(f"guess1 is out at t = {t}, error is {abs(line_true_pos - line_guess1_pos)}")
        if abs(line_true_pos - line_guess2_pos) > measure_error:
            print(f"guess2 is out at t = {t}, error is {abs(line_true_pos - line_guess2_pos)}")


guess1 is out at t = 30, error is 90.00000000000011
guess2 is out at t = 30, error is 90.0
guess1 is out at t = 40, error is 160.00000000000023
guess2 is out at t = 40, error is 160.0
guess1 is out at t = 50, error is 250.0
guess2 is out at t = 50, error is 250.0
guess1 is out at t = 60, error is 360.00000000000045
guess2 is out at t = 60, error is 360.0
guess1 is out at t = 70, error is 490.0
guess2 is out at t = 70, error is 490.0
guess1 is out at t = 80, error is 640.0000000000009
guess2 is out at t = 80, error is 640.0
guess1 is out at t = 90, error is 810.0
guess2 is out at t = 90, error is 810.0


In [2]:
def AddVectors(*ls, subtract = False):
    """
    *ls: list of vectors to add together

    returns the resulting vector sum
    """
    ls_out = [0]*len(ls[0])
    for l in ls:
        k = 0
        for v in l:
            if not subtract:
                ls_out[k] += v
            else:
                ls_out[k] -= v
            k += 1
    return ls_out

In [3]:
class Pendulum:

    # initiate starting position and constant parameters
    def __init__(self, pivot = [0,0], angle = pi/6.0, mass = 1, length = 1, angular_vel = 0):
        """
        pivot: location of pivot in cartesian coordinates
        angle: starting angle for the pendulum (radians?)
        mass: mass at the end of the pendulum
        length: length of the pendulum
        angular_vel: starting angular velocity
        """
        
        self.g = [0,1]
        self.mass = mass
        self.angle = angle # measured clockwise from downard y axis
        self.angular_vel = angular_vel
        self.angular_acc = 0
        self.length = length
        self.pivot = pivot
        self.position = self.get_position()
        
    # Calculate position (x,y) of the pendulum
    def get_position(self):
        """
        return the cartesian coordinates of the end point of the pendulum
        """
        self.position = [self.pivot[0] + self.length*sin(self.angle), 
                         self.pivot[1] - self.length*cos(self.angle)]
        
        if self.angle == pi/2.0 or self.angle == -pi/2.0: self.position[1] = self.pivot[1] 
        if self.angle == 0 or self.angle == pi: self.position[0] = self.pivot[0] 
        
        return self.position
    
    def get_angle(self):
        """
        return the angle of the pendulum
        """
        return self.angle

    def update_angle(self):
        """
        Euler integration to get angle from acceleration
        """
        self.angular_vel += self.angular_acc
        self.angle += self.angular_vel
        
    def apply_force(self, force, *force_args, **force_kwargs):
        """
        Apply arbitrary force
        """
        if not force: return
        
        if force_args or force_kwargs:
            force = force(*force_args, **force_kwargs)
        force[:] = [i / self.mass for i in force]
        
        self.angular_acc += np.arctan2(force[1], force[0])

    def force_from_pivot(self, g):
        """
        g: strength of gravity

        return the forces acting on a pendulum from connected pivot
        """
        return [-self.mass*self.g[0]*cos(self.angle)*sin(self.angle), 
                -self.mass*self.g[0]*cos(self.angle)*cos(self.angle)]
    
    def force_on_pivot(self, g):
        """
        g: strength of gravity
        
        return the force acting on the pivot from the pendulum
        """
        # f = force_from_pivot(g)
        # f = [-i for i in f]
        # return f
        return [self.mass*self.g[0]*cos(self.angle)*sin(self.angle), 
                self.mass*self.g[0]*cos(self.angle)*cos(self.angle)]

    # Print data about pendulum
    def print_status(self, all = False):
        """
        all: whether to print all attributes

        print the attributes of the pendulum
        """
        if all:
            print("mass is %d"% (self.mass))
            print("length is %d"% (self.length))
            print("pivot is %d"% (self.pivot))
        print("angle is %d"% (self.angle))
        print("angular velocity is %d"% (self.angular_vel))


In [4]:
class Double_Pend():
    def __init__(self):
        self.origin = (0,0)
        self.p1 = Pendulum(self.origin, angle = pi/6)
        self.p2 = Pendulum(self.p1.get_position(), angle = pi/3)
        self.position_record = []
        self.angle_record = []
        self.position_record_intermittent = []
        self.angle_record_intermittent = []

    def get_ang_acc(self, g=[0,-1]):
        """
        from the position of each pendulum, calculate the angular acceleration
        """
        # TODO one should be p1 the other should be p2 - which is which?
        num1 = -g[1]*(2*self.p1.mass + self.p2.mass)*sin(self.p1.angle)
        num2 = -self.p2.mass*g[1]*sin(self.p1.angle-2*self.p2.angle)
        num3 = -2*self.p2.mass*sin(self.p1.angle-self.p2.angle) 
        num4 = self.p2.angular_vel**2 * self.p2.length + self.p1.angular_vel**2 * self.p1.length * cos(self.p1.angle - self.p2.angle)
        den = 2*self.p1.mass + self.p2.mass*(1-cos(2*(self.p1.angle-self.p2.angle)))

        self.p1.angular_acc = (num1 + num2 + num3*num4)/(self.p1.length*den)
        
        num1 = 2*sin(self.p1.angle-self.p2.angle)
        num2 = self.p1.angular_vel**2 * self.p1.length * (self.p1.mass + self.p2.mass)
        num3 = g[1]*(self.p1.mass+self.p2.mass)*cos(self.p1.angle)
        num4 = self.p2.angular_vel**2 * self.p2.length * self.p2.mass * cos(self.p1.angle - self.p2.angle)

        self.p1.angular_acc = num1*(num2 + num3 + num4)/(self.p2.length*den)


    def copy_modify(self, angle_diff = [0,0]):
        """
        angle_diff: how much to change the angles for the new pendulum

        copy and edit a double pendulum
        """
        #copy step
        copy = Double_Pend()
        copy.origin = self.origin
        copy.p1 = self.p1
        copy.p2 = self.p2

        # modify step
        copy.p1.angle += angle_diff[0]
        copy.p2.angle += angle_diff[1]

        return copy
    
    def get_angle(self):
        return [self.p1.get_angle(). self.p2.get_angle()]

    def record_position(self):
        self.position_record.append((self.p1.get_position(), self.p2.get_position()))
    
    def record_angle(self):
        self.angle_record.append((self.p1.get_angle(), self.p2.get_angle()))

    def record_to_file(self):
        pass
    
    def get_position_record(self, idx=None, intermittent=False):
        if not intermittent:
            out = self.position_record
            if idx:
                out = [p for i,p in enumerate(self.position_record) if i in idx]
        else:
            out = self.position_record_intermittent
            if idx:
                out = [p for i,p in enumerate(self.position_record_intermittent) if i in idx]
        return out

    def get_angle_record(self, idx=None, intermittent=False):
        if not intermittent:
            out = self.angle_record
            if idx:
                out = [p for i,p in enumerate(self.angle_record) if i in idx]
        else:
            out = self.angle_record_intermittent
            if idx:
                out = [p for i,p in enumerate(self.angle_record_intermittent) if i in idx]
        return out


In [ ]:
d = Double_Pend()
duration = dur
g = [0,-1]
records = {}

def display(sim):
    window_dim = 2*max(d.p1.length, d.p2.length)
    fig = plt.figure()
    ax = plt.axes(xlim=(window_dim,-window_dim), ylim=(window_dim,-window_dim))
    line, = ax.plot([],[], lw=2)
    init = lambda : line.set_data([],[])
    animate = lambda i: line.set_data(self.d.get_position_record(idx=i))

def step_fwd_analytic(dp, g):
    dp.get_ang_acc(g)
    dp.p1.update_angle()
    dp.p2.update_angle()
    dp.p2.pivot = dp.p1.get_position()

def step_bwd_analytic(dp, g):
    dp.get_ang_acc(g)
    dp.p1.update_angle()
    dp.p2.update_angle()
    dp.p2.pivot = dp.p1.get_position()

def step_fwd_numeric(dp, g):

    # Apply gravitational and reaction forces
    dp.p2.apply_force(g)
    dp.p1.apply_force(g)
    dp.p2.apply_force(dp.p1.force_from_pivot(g))
    dp.p1.apply_force(dp.p2.force_on_pivot(g))

    # Update angle from acc
    dp.p1.update_angle()
    dp.p2.update_angle()

    # ensure that pivots are in correct places
    dp.p1.pivot = [0,0]
    dp.p2.pivot = dp.p1.get_position()

def step_bwd_numeric(dp, g):

    # Apply gravitational and reaction forces
    dp.p2.apply_force(g)
    dp.p1.apply_force(g)
    dp.p2.apply_force(dp.p1.force_from_pivot(g))
    dp.p1.apply_force(dp.p2.force_on_pivot(g))

    # Update angle from acc
    dp.p1.update_angle()
    dp.p2.update_angle()

    # ensure that pivots are in correct places
    dp.p1.pivot = [0,0]
    dp.p2.pivot = dp.p1.get_position()




# Simulate "true" pendulum first
then compare some subsequent pendula

In [ ]:
sim, measure_time
measure_error
display=False

# simulation of true pendulum
for t in range(self.duration):
    sim(d)
    d.record_angle()
    d.record_position()

# set up error pendulum
start_error = measure_error/2
dp_alt = d.copy_modify([start_error, start_error])
comp_vec = lambda x, y, e: [abs(i-j)>e for i,j in zip(x,y)]

# simulate error pendulum
nth_record = 0
self.records = {start_error:[]}
while t < duration:
    sim(dp_alt)
    dp_alt.record_angle()
    dp_alt.record_position()

    # if error pendulum is wrong
    if t%measure_time == 0:
        if any(comp_vec(dp_alt.get_angle(), d.get_angle_record(idx = nth_record), measure_error)):
            print(f"got to {t} steps with initial error {start_error}")

            self.records[start_error] = dp_alt.get_position_record()
            start_error /= 2
            dp_alt = d.copy_modify([start_error, start_error])
            t=0
            nth_record=0

        nth_record += 1

    t += 1


# set up error pendulum
start_error = -start_error
dp_alt2 = d.copy_modify([start_error, start_error])

# simulate error pendulum
nth_record = 0
while t < duration:
    sim(dp_alt2)

    # if error pendulum is wrong
    if t%measure_time == 0:
        if any(comp_vec(dp_alt2.get_angle(), d.get_angle_record(idx = nth_record), measure_error)):
            print(f"got to {t} steps with initial error {start_error}")

            records[start_error] = dp_alt.get_position_record()
            start_error /= 2
            dp_alt = d.copy_modify([start_error, start_error])
            t=0
            nth_record=0

        nth_record += 1

    t += 1

records[0] = d.get_position_record()

# Simulation methods based on detecting exactly when second pendulum goes outside error bound

In [ ]:
"""
sim: whether to solve numerically or analytically
measure_time: how often to take measurements
measure_error: error range in each measurement
n_pends: number of pendulums to simulate

simulate the error, and take measurements at regular time intervals
"""

sim = step_fwd_numeric
measure_time = 100
measure_error = pi/100
n_pends = 10

dp_true = d
dps = []
for i in range(n_pends):
    dps.append(d)
    dps[i] = dps[i].copy_modify([0, measure_error*i/n_pends])
ref = n_pends//2

# List for recording large errors
# ???

# Run simulation
for t in range(duration):

    # Simulate
    sim(dp_true)
    for i,p in enumerate(dps):
        sim(dps[i])

    # Check timing intervals
    if t % measure_time == 0:

        # Record positions
        for i,d in enumerate(dps):
            self.d.record_angle()

        # Check if out of error bounds
        for i,d in enumerate(dps):
            if abs(dps[i].p1.position[1] - dps[0].p1.position[1]) > measure_error:
                pass


# Measure at periodic time intervals

In [ ]:
"""
sim: whether to solve numerically or analytically
measure_time: how often to take measurements
measure_error: error range in each measurement
n_pends: number of pendulums to simulate

simulate the error, and take measurements at regular time intervals
"""
sim = step_fwd_numeric
measure_time = 100
measure_error = pi/100
n_pends = 10

dp_true = d
dp_error = d.copy_modify([0,0])

# List for recording large errors
# ???

# Run simulation
for t in range(duration):

    # Simulate
    sim(dp_true)
    sim(dp_error)

    # Check timing intervals
    if t % measure_time == 0:

        # Record positions
        dp_true.record_angle()
        dp_error.record_angle()

        # Check if out of error bounds
        if abs(dp_true.p1.position[1] - dp_error.p1.position[1]) > measure_error:
            pass

In [ ]:

sim = step_fwd_numeric
measure_time = 100
measure_error = pi/100
n_pends = 10
error_threshold = 0.5
dps = []
for i in range(-n_pends/2,0):
    dps.append(d)
    dps[i] = dps[i].copy_modify([0, measure_error*i/n_pends])

for i in range(1,n_pends/2+1):
    dps.append(d)
    dps[i] = dps[i].copy_modify([0, measure_error*i/n_pends])

for t in range(duration):

    # Simulate
    sim(d)
    for i in enumerate(dps):
        sim(dps[i])

    # Record positions
    if t%measure_time == 0:
        d.record_angle()
        for i in enumerate(dps):
            dps[i].record_angle()

        #find errors
        for i in enumerate(dps):
            error = [0,0]

            # use angles
            d.p1.get_angle() - dps[i].p1.get_angle()
            d.p2.get_angle() - dps[i].p2.get_angle()

            if error > measure_error:
                # get initial deviance of pendulum
                # create new pendulums
                pass



        #
        dps = []
        for i in range(-n_pends/2,0):
            dps.append(self.d)
            dps[i] = dps[i].copy_modify([0, error_threshold*i/n_pends])

        for i in range(1,n_pends/2+1):
            dps.append(self.d)
            dps[i] = dps[i].copy_modify([0, error_threshold*i/n_pends])

In [ ]:

"""
sim: a function for how to calculate one step of the simulation
measure_error: error range in each measurement
error_threshold: error at which to start simulating a closer pendulum

simulate the error, and take measurements at regular time intervals
"""

sim = step_fwd_numeric
measure_error = pi/100
n_pends = 2, 
error_threshold = pi/20.0

dp_true = d
dp_neg = d.copy_modify([-measure_error/2, -measure_error/2])
dp_pos = d.copy_modify([measure_error/2, measure_error/2])

for t in range(self.duration):

    sim(dp_true)
    sim(dp_neg)
    sim(dp_pos)

    # Record positions
    dp_true.record_angle()
    dp_neg.record_angle()
    dp_pos.record_angle()

    # calculate errors in angles
    pos_error = [0,0] 
    pos_error[0] += dp_true.p1.get_angle() - dp_pos.p1.get_angle()
    pos_error[1] += dp_true.p2.get_angle() - dp_pos.p2.get_angle()

    neg_error = [0,0] 
    neg_error[0] += dp_true.p1.get_angle() - dp_neg.p1.get_angle()
    neg_error[1] += dp_true.p2.get_angle() - dp_neg.p2.get_angle()

    if any(pos_error > error_threshold):
        # record t, with note of initial conditions

        # reinitialize dp_pos with smaller error
        dp_pos = dp_pos/2 # initial state of true with smaller error deviation

        # simulate dp_pos up to current time (still checking errors
        for u in range(t):
            sim(dp_pos)

    if any(neg_error > error_threshold):
        # record t, with note of initial conditions

        # reinitialize dp_neg with smaller error
        dp_neg = dp_neg/2# initial state of true with smaller error deviation

        # simulate dp_neg up to current time (still checking errors)
        for u in range(t):
            sim(dp_neg)


In [ ]:
class Simulation():
    def __init__(self, dur = 7000):
        self.d = Double_Pend()
        self.duration = dur
        self.g = [0,-1]
        self.records = {}
    
    def display(self, sim):
        window_dim = 2*max(self.d.p1.length, self.d.p2.length)
        fig = plt.figure()
        ax = plt.axes(xlim=(window_dim,-window_dim), ylim=(window_dim,-window_dim))
        line, = ax.plot([],[], lw=2)
        init = lambda : line.set_data([],[])
        animate = lambda i: line.set_data(self.d.get_position_record(idx=i))
        
        animation = ani.FuncAnimation(fig, animate, init_func=init, frames=200, interval=20, blit=True)
        plt.show()

    def step_fwd_analytic(self, dp):
        dp.get_ang_acc(self.g)
        dp.p1.update_angle()
        dp.p2.update_angle()
        dp.p2.pivot = self.d.p1.get_position()

    def step_bwd_analytic(self, dp):
        dp.get_ang_acc(self.g)
        dp.p1.update_angle()
        dp.p2.update_angle()
        dp.p2.pivot = self.d.p1.get_position()

    def step_fwd_numeric(self, dp):
            
        # Apply gravitational and reaction forces
        dp.p2.apply_force(self.g)
        dp.p1.apply_force(self.g)
        dp.p2.apply_force(dp.p1.force_from_pivot(self.g))
        dp.p1.apply_force(dp.p2.force_on_pivot(self.g))
        
        # Update angle from acc
        dp.p1.update_angle()
        dp.p2.update_angle()

        # ensure that pivots are in correct places
        dp.p1.pivot = [0,0]
        dp.p2.pivot = self.d.p1.get_position()

    def step_bwd_numeric(self, dp):
            
        # Apply gravitational and reaction forces
        dp.p2.apply_force(self.g)
        dp.p1.apply_force(self.g)
        dp.p2.apply_force(dp.p1.force_from_pivot(self.g))
        dp.p1.apply_force(dp.p2.force_on_pivot(self.g))
        
        # Update angle from acc
        dp.p1.update_angle()
        dp.p2.update_angle()

        # ensure that pivots are in correct places
        dp.p1.pivot = [0,0]
        dp.p2.pivot = self.d.p1.get_position()

    def simulate_error_split(self, sim = step_fwd_numeric, measure_error = pi/100, n_pends = 2, error_threshold = pi/20.0):
        """
        sim: a function for how to calculate one step of the simulation
        measure_error: error range in each measurement
        error_threshold: error at which to start simulating a closer pendulum

        simulate the error, and take measurements at regular time intervals
        """
        

        dp_true = self.d
        dp_neg = self.d.copy_modify([-measure_error/2, -measure_error/2])
        dp_pos = self.d.copy_modify([measure_error/2, measure_error/2])
    
        for t in range(self.duration):
            
            sim(dp_true)
            sim(dp_neg)
            sim(dp_pos)
            
            # Record positions
            dp_true.record_angle()
            dp_neg.record_angle()
            dp_pos.record_angle()
                    
            # calculate errors in angles
            pos_error = [0,0] 
            pos_error[0] += dp_true.p1.get_angle() - dp_pos.p1.get_angle()
            pos_error[1] += dp_true.p2.get_angle() - dp_pos.p2.get_angle()
                
            neg_error = [0,0] 
            neg_error[0] += dp_true.p1.get_angle() - dp_neg.p1.get_angle()
            neg_error[1] += dp_true.p2.get_angle() - dp_neg.p2.get_angle()
            
            if any(pos_error > error_threshold):
                # record t, with note of initial conditions

                # reinitialize dp_pos with smaller error
                dp_pos = dp_pos/2 # initial state of true with smaller error deviation
                
                # simulate dp_pos up to current time (still checking errors
                for u in range(t):
                    sim(dp_pos)

            if any(neg_error > error_threshold):
                # record t, with note of initial conditions

                # reinitialize dp_neg with smaller error
                dp_neg = dp_neg/2# initial state of true with smaller error deviation
                
                # simulate dp_neg up to current time (still checking errors)
                for u in range(t):
                    sim(dp_neg)

    def simulate_time_split2(self, sim = step_fwd_numeric, measure_time = 100, measure_error = pi/100, n_pends = 10):
        """
        sim: whether to solve numerically or analytically
        measure_time: how often to take measurements
        measure_error: error range in each measurement
        n_pends: number of pendulums to simulate

        simulate the error, and take measurements at regular time intervals
        """
        
        dp_true = self.d
        dp_error = self.d.copy_modify([0,0])
            
        # List for recording large errors
        # ???

        # Run simulation
        for t in range(self.duration):
            
            # Simulate
            sim(dp_true)
            sim(dp_error)
            
            # Check timing intervals
            if t % measure_time == 0:
                
                # Record positions
                dp_true.record_angle()
                dp_error.record_angle()
                
                # Check if out of error bounds
                if abs(dp_true.p1.position[1] - dp_error.p1.position[1]) > measure_error:
                    pass

    def simulate_time_split(self, sim = step_fwd_numeric, measure_time = 100, measure_error = pi/100, n_pends = 10):
        """
        sim: whether to solve numerically or analytically
        measure_time: how often to take measurements
        measure_error: error range in each measurement
        n_pends: number of pendulums to simulate

        simulate the error, and take measurements at regular time intervals
        """
        
        dp_true = self.d
        dps = []
        for i in range(n_pends):
            dps.append(self.d)
            dps[i] = dps[i].copy_modify([0, measure_error*i/n_pends])
        ref = n_pends//2
            
        # List for recording large errors
        # ???

        # Run simulation
        for t in range(self.duration):
            
            # Simulate
            sim(dp_true)
            for i,p in enumerate(dps):
                sim(dps[i])
            
            # Check timing intervals
            if t % measure_time == 0:
                
                # Record positions
                for i,d in enumerate(dps):
                    self.d.record_angle()
                
                # Check if out of error bounds
                for i,d in enumerate(dps):
                    if abs(dps[i].p1.position[1] - dps[0].p1.position[1]) > measure_error:
                        pass

    def simulate_time_split3(self, sim = step_fwd_numeric, measure_time = 100, measure_error = pi/100, n_pends = 10, error_threshold = 0.5):
        
        dps = []
        for i in range(-n_pends/2,0):
            dps.append(self.d)
            dps[i] = dps[i].copy_modify([0, measure_error*i/n_pends])
            
        for i in range(1,n_pends/2+1):
            dps.append(self.d)
            dps[i] = dps[i].copy_modify([0, measure_error*i/n_pends])
        
        for t in range(self.duration):
            
            # Simulate
            sim(self.d)
            for i in enumerate(dps):
                sim(dps[i])

            # Record positions
            if t%measure_time == 0:
                self.d.record_angle()
                for i in enumerate(dps):
                    dps[i].record_angle()
                    
                #find errors
                for i in enumerate(dps):
                    error = [0,0]
                    
                    # use angles
                    self.d.p1.get_angle() - dps[i].p1.get_angle()
                    self.d.p2.get_angle() - dps[i].p2.get_angle()
                    
                    if error > measure_error:
                        # get initial deviance of pendulum
                        # create new pendulums
                        pass
                    
                    
                    
                #
                dps = []
                for i in range(-n_pends/2,0):
                    dps.append(self.d)
                    dps[i] = dps[i].copy_modify([0, error_threshold*i/n_pends])
            
                for i in range(1,n_pends/2+1):
                    dps.append(self.d)
                    dps[i] = dps[i].copy_modify([0, error_threshold*i/n_pends])

    def simulate_no_split(self, sim, measure_time, measure_error, display=False):

        # simulation of true pendulum
        for t in range(self.duration):
            sim(self.d)
            self.d.record_angle()
            self.d.record_position()

        # set up error pendulum
        start_error = measure_error/2
        dp_alt = self.d.copy_modify([start_error, start_error])
        comp_vec = lambda x, y, e: [abs(i-j)>e for i,j in zip(x,y)]

        # simulate error pendulum
        nth_record = 0
        self.records = {start_error:[]}
        while t < self.duration:
            sim(dp_alt)
            dp_alt.record_angle()
            dp_alt.record_position()

            # if error pendulum is wrong
            if t%measure_time == 0:
                if any(comp_vec(dp_alt.get_angle(), self.d.get_angle_record(idx = nth_record), measure_error)):
                    print(f"got to {t} steps with initial error {start_error}")

                    self.records[start_error] = dp_alt.get_position_record()
                    start_error /= 2
                    dp_alt = self.d.copy_modify([start_error, start_error])
                    t=0
                    nth_record=0

                nth_record += 1

            t += 1


        # set up error pendulum
        start_error = -start_error
        dp_alt2 = self.d.copy_modify([start_error, start_error])

        # simulate error pendulum
        nth_record = 0
        while t < self.duration:
            sim(dp_alt2)

            # if error pendulum is wrong
            if t%measure_time == 0:
                if any(comp_vec(dp_alt2.get_angle(), self.d.get_angle_record(idx = nth_record), measure_error)):
                    print(f"got to {t} steps with initial error {start_error}")
                    
                    self.records[start_error] = dp_alt.get_position_record()
                    start_error /= 2
                    dp_alt = self.d.copy_modify([start_error, start_error])
                    t=0
                    nth_record=0

                nth_record += 1

            t += 1
        
        self.records[0] = self.d.get_position_record()

